In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path

In [2]:
""" CREATE CONTEXT PD's """
def test_get_context_tiles(pd_data, tile_coord, context_size):
    assert context_size % 2 == 1, 'context size must be and odd number'
    context_range = int((context_size - 1) / 2)
    
    context_tile_coords = []
    for x in range(-context_range, context_range+1):
        for y in range(-context_range, context_range+1):
            context_tile_coords.append((tile_coord[0]+x,tile_coord[1]+y) in pd_data.index)
    return np.all(context_tile_coords)

In [3]:
def generate_context(col, row):
    col_rows = []
    for x in range(-1,2):
        for y in range(-1,2):
            col_rows.append((col+x, row+y))
    return col_rows

In [4]:
def filter_macrotiles(input_dir, output_dir, context_size):
    
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
        
    slides = list(input_dir.glob('*.gz'))
    
    for slide_num, _fn in enumerate(slides):
        print('Opening: {}'.format(_fn))
        print('{}/{}'.format(slide_num, len(slides)))
        _pd_data = pd.read_pickle(str(_fn))
        boolean_mask = []
        for row_idx, row in enumerate(_pd_data.iterrows()):
            if row_idx % 5000 == 0:
                print('{:,}/{:,}\r'.format(row_idx, len(_pd_data)), end='')
            boolean_mask.append(test_get_context_tiles(_pd_data, row[0], context_size))
        _macro_tile_pd = _pd_data[boolean_mask]
        print('{} | {} -> {} ({:.2%} reduction.)'.format(_fn.stem, len(_pd_data), len(_macro_tile_pd), len(_macro_tile_pd)/len(_pd_data)))
        _macro_tile_pd.to_pickle(str(output_dir / '{}-context{}.gz'.format(_fn.stem, context_size)))

In [8]:
def test_purity_context_tiles(df, tile_coord, context_size):
    assert context_size % 2 == 1, 'context size must be and odd number'
    context_range = int((context_size - 1) / 2)
    
    context_tile_coords = []
    
    context_tiles = [(tile_coord[0]+x, tile_coord[1]+y) for x in range(-context_range, context_range+1) for y in range(-context_range, context_range+1)]
    context_tiles_classes = df.loc[context_tiles]['class'].ravel()
    
    if np.isnan(context_tiles_classes).any():
        # Not a valid macrotile
        return False, False
    
    is_pure = len(np.unique(context_tiles_classes)) == 1
    return True, is_pure

def filter_pure_macrotiles(input_dir, output_dir, context_size):
    
    for dir_name in ['pure', 'mixed']:
        if not (output_dir / dir_name).exists():
            (output_dir / dir_name).mkdir(parents=True)
        
    slides = list(input_dir.glob('*.gz'))
    
    for slide_num, _fn in enumerate(slides):
        
        print('[{:2}/{:2}] Filtering {}'.format(slide_num, len(slides), _fn.stem))
        df = pd.read_pickle(str(_fn))
        is_macrotile_valid = []
        is_macrotile_pure = []
        
        for row_idx, row in enumerate(df.iterrows()):
            if row_idx % 5000 == 0:
                print('{:,}/{:,}\r'.format(row_idx, len(df)), end='')
            is_valid, is_pure = test_purity_context_tiles(df, row[0], context_size)
            is_macrotile_valid.append(is_valid)
            is_macrotile_pure.append(is_pure)
        
        is_macrotile_valid = np.array(is_macrotile_valid)
        is_macrotile_pure = np.array(is_macrotile_pure)
            
        _mixed_macrotiles_pd = df[is_macrotile_valid & np.logical_not(is_macrotile_pure)]
        _pure_macrotiles_pd = df[is_macrotile_valid & is_macrotile_pure]
        
        if len(_mixed_macrotiles_pd) > 0:
            _mixed_macrotiles_pd.to_pickle(str(output_dir / 'mixed' / '{}-context{}.gz'.format(_fn.stem, context_size)))
        
        if len(_pure_macrotiles_pd) > 0:
            _pure_macrotiles_pd.to_pickle(str(output_dir / 'pure' / '{}-context{}.gz'.format(_fn.stem, context_size)))
        

In [9]:
if __name__=='__main__':
    DIR_PATH = Path('/home/matejg/Project/crc_ml/data/processed/Prostata/level1/')
    IN_DIR = DIR_PATH / 'tiles'
    OUT_DIR = DIR_PATH
    CONTEXT_SIZE = 3        

    filter_pure_macrotiles(IN_DIR, OUT_DIR, CONTEXT_SIZE)   

[ 0/133] Filtering P-2019_1399-02-0-bg4-tile96


/home/matejg/Test/test/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


[ 1/133] Filtering P-2019_3204-08-0-bg4-tile96


/home/matejg/Test/test/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


[ 2/133] Filtering P-2019_4624-07-0-bg4-tile96


...

/home/matejg/Test/test/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


[131/133] Filtering P-2019_4002-05-0-bg4-tile96


/home/matejg/Test/test/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


[132/133] Filtering P-2019_2781-11-1-bg4-tile96


/home/matejg/Test/test/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


In [7]:
total = 0
dir_name = 'mixed'
for _fn in (OUT_DIR / dir_name).glob('*.gz'):
    df = pd.read_pickle(str(_fn))
    print('{}: {:,}'.format(_fn.stem, len(df)))
    total += len(df)
print('Total: {:,} {} macrotiles.'.format(total, dir_name))

P-2019_4336-11-1-bg4-tile96-context3: 548
Total: 548 mixed macrotiles.
